In [1]:
# Problem Statement:
# Given a partial potential knights tour, can we QUICKLY fill it out?
# 16 squares will be labeled with the position
import numpy as np
from numpy import uint8, int8
from itertools import permutations, combinations
from random import randint

In [2]:
C = 601080390 #how many combinations of 32, 16
P = 20922789888000 # 16 !
#given these two numbers, choose 16 of the 32 blocks, and name them accordingly

def fact(n):
    return 1 if n < 1 else n * fact(n-1)

def choose(n, k):
    return fact(n)//(fact(k) * fact(n-k)) if n >= k else 0

'''https://en.wikipedia.org/wiki/Combinatorial_number_system'''
def enumerate_comb(c, n, k):
    #c = enumeration, n = total number, k = how many chosen
    l = []
    for den in range(k, 0, -1):
#         print(den)
        #count down from k
        for num in range(0, n+1, 1):
#             print("\t{}".format(num), end=": ")
            #count up the c1...ck
            ch = choose(n-num, den)
#             print(ch)
            if ch <= c:
                l.append(num - 1) #0 based indexing
                c -= choose(n-num, den)
                break;
    return tuple(l)

def enumerate_perm(p, n):
    nums = []
    for i in range(n, 0, -1):
        number_of_nodes_in_layer = i #how many items are in the layer we are on
        size_of_node_in_layer = fact(i-1)
        nums.append(p//size_of_node_in_layer)
        p = p%size_of_node_in_layer
    vals = list(range(n))
    permutation = []
    for n in nums:
        permutation.append(vals.pop(n))
#     print(nums)
#     print(vals)
#     print(permutation)
#     print()
    return tuple(permutation)

In [3]:
# Test that the enumerate_<perm|comb> methods are working as intended
perms = []
for i in range(fact(7)):
    perms.append(enumerate_perm(i, 7))
combs = []
for i in range(choose(14,6)):
    combs.append(enumerate_comb(i, 14, 6))

lp = list(permutations(range(7))) 
lc = list(combinations(range(14),6))

for i in perms:
    assert i in lp
for i in lp:
    assert i in perms
for i in combs:
    assert i in lc
for i in lc:
    assert i in combs

In [4]:
def get_rand_init():
    p = enumerate_perm(randint(0,P-1), 16)
    c = enumerate_comb(randint(0,C-1), 32, 16)
    for j in range(16):
        assert j in p
    for c_item in c:
        assert c_item < 32 and c_item >= 0
        assert c.count(c_item) == 1
    return gen_board_from_pc(p,c)

def gen_board_from_pc(perm, comb):
    board = np.ones(64, dtype=int8) * -1
    black_squares = [0,2,4,6,9,11,13,15,16,18,20,22,25,27,29,31,32,32,36,38,41,43,45,47,48,50,52,54,57,59,61,63]
    for idx in range(16):
        board[black_squares[comb[idx]]] = perm[idx]*4
    return board

def print_board(board):
    print('------------------------')
    for i in range(8):
        for j in range(8):
            num = board[i*8+j]
            if num != -1:
                print(str(num).ljust(2), end = ' ')
            else:
                print('  ', end = ' ')
        print()
    print('------------------------')

In [5]:
for i in range(5):
    print_board(get_rand_init())

------------------------
60    40    56    24    
   16                12 
      32          44    
                        
8                       
   52                   
0           4     36    
   28    20          48 
------------------------
------------------------
                        
   20    52          4  
                  36    
   24          60    28 
56                40    
   0     32             
44    48          16    
   12          8        
------------------------
------------------------
48                      
   12          28    36 
      40                
         52    56       
4                 20    
                     0  
8     60          32    
         44    16       
------------------------
------------------------
52    36          56    
         24             
            20          
   12    48          4  
8                       
         32          60 
0     40          28    
   16                44 
------------------------


In [6]:
kng = np.array([
    [17, 10, -1, -1, -1, -1, -1, -1],[18, 11, 16, -1, -1, -1, -1, -1],
    [19, 12,  8, 17, -1, -1, -1, -1],[20, 13,  9, 18, -1, -1, -1, -1],
    [21, 14, 10, 19, -1, -1, -1, -1],[22, 15, 11, 20, -1, -1, -1, -1],
    [23, 12, 21, -1, -1, -1, -1, -1],[13, 22, -1, -1, -1, -1, -1, -1],
    [25, 18,  2, -1, -1, -1, -1, -1],[26, 19,  3, 24, -1, -1, -1, -1],
    [27, 20,  4,  0, 16, 25, -1, -1],[28, 21,  5,  1, 17, 26, -1, -1],
    [29, 22,  6,  2, 18, 27, -1, -1],[30, 23,  7,  3, 19, 28, -1, -1],
    [31,  4, 20, 29, -1, -1, -1, -1],[ 5, 21, 30, -1, -1, -1, -1, -1],
    [33, 26, 10,  1, -1, -1, -1, -1],[34, 27, 11,  2,  0, 32, -1, -1],
    [35, 28, 12,  3,  1,  8, 24, 33],[36, 29, 13,  4,  2,  9, 25, 34],
    [37, 30, 14,  5,  3, 10, 26, 35],[38, 31, 15,  6,  4, 11, 27, 36],
    [39,  7,  5, 12, 28, 37, -1, -1],[ 6, 13, 29, 38, -1, -1, -1, -1],
    [41, 34, 18,  9, -1, -1, -1, -1],[42, 35, 19, 10,  8, 40, -1, -1],
    [43, 36, 20, 11,  9, 16, 32, 41],[44, 37, 21, 12, 10, 17, 33, 42],
    [45, 38, 22, 13, 11, 18, 34, 43],[46, 39, 23, 14, 12, 19, 35, 44],
    [47, 15, 13, 20, 36, 45, -1, -1],[14, 21, 37, 46, -1, -1, -1, -1],
    [49, 42, 26, 17, -1, -1, -1, -1],[50, 43, 27, 18, 16, 48, -1, -1],
    [51, 44, 28, 19, 17, 24, 40, 49],[52, 45, 29, 20, 18, 25, 41, 50],
    [53, 46, 30, 21, 19, 26, 42, 51],[54, 47, 31, 22, 20, 27, 43, 52],
    [55, 23, 21, 28, 44, 53, -1, -1],[22, 29, 45, 54, -1, -1, -1, -1],
    [57, 50, 34, 25, -1, -1, -1, -1],[58, 51, 35, 26, 24, 56, -1, -1],
    [59, 52, 36, 27, 25, 32, 48, 57],[60, 53, 37, 28, 26, 33, 49, 58],
    [61, 54, 38, 29, 27, 34, 50, 59],[62, 55, 39, 30, 28, 35, 51, 60],
    [63, 31, 29, 36, 52, 61, -1, -1],[30, 37, 53, 62, -1, -1, -1, -1],
    [58, 42, 33, -1, -1, -1, -1, -1],[59, 43, 34, 32, -1, -1, -1, -1],
    [60, 44, 35, 33, 40, 56, -1, -1],[61, 45, 36, 34, 41, 57, -1, -1],
    [62, 46, 37, 35, 42, 58, -1, -1],[63, 47, 38, 36, 43, 59, -1, -1],
    [39, 37, 44, 60, -1, -1, -1, -1],[38, 45, 61, -1, -1, -1, -1, -1],
    [50, 41, -1, -1, -1, -1, -1, -1],[51, 42, 40, -1, -1, -1, -1, -1],
    [52, 43, 41, 48, -1, -1, -1, -1],[53, 44, 42, 49, -1, -1, -1, -1],
    [54, 45, 43, 50, -1, -1, -1, -1],[55, 46, 44, 51, -1, -1, -1, -1],
    [47, 45, 52, -1, -1, -1, -1, -1],[46, 53, -1, -1, -1, -1, -1, -1]
], dtype=int8)

degree = np.array([
    2,3,4,4,4,4,3,2,
    3,4,6,6,6,6,4,3,
    4,6,8,8,8,8,6,4,
    4,6,8,8,8,8,6,4,
    4,6,8,8,8,8,6,4,
    4,6,8,8,8,8,6,4,
    3,4,6,6,6,6,4,3,
    2,3,4,4,4,4,3,2
], dtype=int8)

order = np.array([0,4,8,12,16,20,24,32,36,40,44,48,52,56,60,0], dtype=int8)

translate = np.array([
	'@','0','1','2','3','4','5','6','7','8','9','a','b','c','d','e',
	'f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u',
	'v','w','x','y','z','+','A','B','C','D','E','F','G','H','I','J',
	'K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
])

visited = np.array([
    -1,-1,-1,-1,-1,-1,-1,-1,
    -1,-1,-1,-1,-1,-1,-1,-1,
    -1,-1,-1,-1,-1,-1,-1,-1,
    -1,-1,-1,-1,-1,-1,-1,-1,
    -1,-1,-1,-1,-1,-1,-1,-1,
    -1,-1,-1,-1,-1,-1,-1,-1,
    -1,-1,-1,-1,-1,-1,-1,-1,
    -1,-1,-1,-1,-1,-1,-1,-1
], dtype=int8)